# Exercise 1: Collaborative Filtering & Evaluation

*Learning from User-generated Data, CP MMS, JKU Linz 2021*

This exercise is comprised of two parts: **Collaborative filtering** and **Recommender Systems Evaluation I**.\
It concerns material of Lecture 2 "**Memory-based Collaborative Filtering (CF)**" and partially Lecture 3 "**Evaluating Recommender Systems**".

The deadline for submitting the whole Exercise 1 is **30.03.2021**.

## Requirements
Below you will find four tasks. Each task is graded on a scale from 0 to 3 points. 12 points is the maximal score possible for this exercise.

Solving each task will require you to edit this Jupyter Notebook. It is to be uploaded to **Moodle** before the deadline. Make sure to enter your name and matriculation number into the file name!\
Example file name: "**Ex-1_Bond-James_k0000007.ipynb**"


If a task requires you to write a function, follow the template provided. Do not change the template and make sure that your function acts in accordance with the specifications.

It should be possible to run all cells of your submitted notebook from the first to the last without errors in a reasonable time (under 2 minutes). In the opposite case you may loose points or even receive zero for the whole exercise, so **be careful with what you submit**.

For this exercise only add new cells and import additional libraries if you really-really need to (you most probably won't need to). **Feel free to experiment with the notebook, but clean it up before submitting.**

---

## Recap: Collaborative Filtering (CF)
### Interaction Matrix
* Every element is a result of interaction between a certain user and an item. In this case - explicit rating on a scale from 1 to 5 (stars ⭐️);
* In a real world sceario such matrix can be recreated from log files of your database (attached to an online shop, for example), where each interaction is represented as a single record in a very long list. This is quite efficient, because usually interaction matrices are very sparce. Why is it so?
* How do we treat missing values? Should we replace them with zeros?
* What interactions can we consider while creating an interaction matrix?


|        | item_1 | item_2 | item_3 | item_4 | item_5 |
| ---    |   ---  |   ---  |   ---  |   ---  |   ---  |
| user_1 | 3 |   | 2 | 3 | 3 |
| user_2 | 4 | 3 | 4 | 3 |   |
| user_3 | 3 | 2 | 1 | 4 | 4 |
| user_4 |   | 5 | 4 | 3 | 1 |
| user_5 | 5 |   | 3 | 4 | **(?)** |

### Recommendation scenario
* We consider basic scenario of recommendation -- predicting missing rating for an item given a user;
* This scenario can be extended. For example if we predict ratings for all items unseen by the user, we'll be able to propose top 10 items to purchase/listen to/watch next (Amazon, Spotify, YouTube);
* The predicted rating is usually hidden from the user and only used internally by the recommender system;


* Input: **User_id**, **Item_id**;
* Output: **Rating_Estimate** (usually continuous, can be truncated to conform to our scale, but we don't bother);

### User-Based CF
* "I haven't rated the item, I have little idea about it";
* "There probably are **other users**, who interacted with the item and rated it";
* "Let us find users who have preferences most similar to mine and construct the predicted rating for me from their ratigs";
* "I am a kind person and tend to give high ratings to items. Other users can have different habits, we need to take this into account (rating bias)";


### Item-Based CF
* "I haven't rated the item, I have little idea about it";
* "But **I rated other items**, maybe some of them are similar to the one in question?";
* "Let us construct the predicted rating from the ratings I gave to other items";
* "We construct the predicted rating from my own ratings, so my bias is taken into account automatically";

# Recommendation in Action
What do we recommend? Books? Movies? Music? Games?

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Interaction matrix: ratings given to items by users
data = {
    'user_1' : {'item_1' : 3, 'item_2' : np.nan, 'item_3' : 2, 'item_4' : 3, 'item_5' : 3},
    'user_2' : {'item_1' : 4, 'item_2' : 3, 'item_3' : 4, 'item_4' : 3, 'item_5' : np.nan},
    'user_3' : {'item_1' : 3, 'item_2' : 2, 'item_3' : 1, 'item_4' : 4, 'item_5' : 4},
    'user_4' : {'item_1' : np.nan, 'item_2' : 5, 'item_3' : 4, 'item_4' : 3, 'item_5' : 1},
    'Active_User' : {'item_1' : 5, 'item_2' : np.nan, 'item_3' : 3, 'item_4' : 4, 'item_5' : np.nan}
}

# Name the items
items = ['RHCP', 'Radiohead', 'Clown_Core', 'Pink_Floyd', 'ACDC']

df = pd.DataFrame(data).T
df.columns = items

df

,RHCP,Radiohead,Clown_Core,Pink_Floyd,ACDC
user_1,3.0,NaN,2.0,3.0,3.0
user_2,4.0,3.0,4.0,3.0,NaN
user_3,3.0,2.0,1.0,4.0,4.0
user_4,NaN,5.0,4.0,3.0,1.0
Active_User,5.0,NaN,3.0,4.0,NaN


## User-Based CF
Find similar users, recommend according to their respective similarity.

### Which rating do we predict?

In [3]:
# Set User-Item to predict score
target_user = 'Active_User'
target_item = 'ACDC'

print("<"+target_user + "> <" + target_item + "> previous value: " + str(df.loc[target_user,target_item]))

# Create a copy of the data to mess with it
df_pred = df.copy(deep = True)

# Set the score to predict to NaN for convenience (It may have beem NaN already)
df_pred.loc[target_user,target_item] = np.nan # NaN value used so far, here we need the prediction
df_pred

<Active_User> <ACDC> previous value: nan


,RHCP,Radiohead,Clown_Core,Pink_Floyd,ACDC
user_1,3.0,NaN,2.0,3.0,3.0
user_2,4.0,3.0,4.0,3.0,NaN
user_3,3.0,2.0,1.0,4.0,4.0
user_4,NaN,5.0,4.0,3.0,1.0
Active_User,5.0,NaN,3.0,4.0,NaN


### Finding top K similar users

In [4]:
# Calculating similarities between users
# because of NaNs we automatically ignore rows and columns with missing ratings
user_similarity = df_pred.T.corr(method='pearson') ## Pearson's correlation
## T: transpose: users become columns and are so keys to work with
## getting a table full of correlation relations within the users
user_similarity[target_user]

user_1         0.866025
user_2         0.000000
user_3         0.654654
user_4        -1.000000
Active_User    1.000000
Name: Active_User, dtype: float64

In [5]:
# Selecting users that have actually rated the target item
potential_neighbors = list(df_pred[df_pred[target_item].notnull()].index) ## ignore users which are not 
## correlated to active user
neighbors = user_similarity.loc[target_user].filter(potential_neighbors) ## potential_neighbors as 
## boolean mask

# Saving weights and Names of Top K users closest to the Target one
top_k_users = 2
closest_users = neighbors.sort_values(ascending=False)[:top_k_users]
closest_users

user_1    0.866025
user_3    0.654654
Name: Active_User, dtype: float64

### Predicting the Rating


In [6]:
sum_weight = closest_users.sum() # weight-normalization term
target_rating_bias = df.loc[target_user].mean()

predicted_score = target_rating_bias # start from user's own bias

for user in closest_users.keys():
    user_rating_bias = df.loc[user].mean()
    
    user_weight = (closest_users[user]/sum_weight)
    user_rating_dev = df.loc[user, target_item] - user_rating_bias
    
    predicted_score += user_weight * user_rating_dev # take other ratings into account, apply weighting
    ## same as on slide pdf page 20
print("Item <" + target_item +
      "> is recommended to the user <" + target_user +
      "> with score " + str(predicted_score) +
      " out of 5")

Item <ACDC> is recommended to the user <Active_User> with score 4.658975830340907 out of 5


## <font color='red'>TASK 1/4</font>: User-Based CF
Compose the above code into a function which returns a single value: predicted score. Consult lecture slides to make sure you don't miss a thing! Use the template below:

In [7]:
def user_based_CF(df: pd.DataFrame, target_user: str, target_item: str, top_k = 2) -> float:
    '''
    df - Data Frame - interation matrix
    target_user - String - user to predict rating for
    target_item - String - item to predict rating for
    top_k - Int - number of neighbors to predict ratings from (default 2)
    
    returns - float - predicted rating 
    '''

    # Create a copy of the data to mess with it
    df_pred = df.copy(deep = True) ## dataframe to work with

    # Set the score to predict to NaN for convenience (It may have beem NaN already)
    df_pred.loc[target_user,target_item] = np.nan ## NaN value used so far, here we  will need the prediction
    ## location of prediction
    
    
    # Calculating similarities between users
    # because of NaNs we automatically ignore rows and columns with missing ratings
    user_similarity = df_pred.T.corr(method='pearson') ## Pearson's correlation
    ## T: transpose: users become columns and are so keys to work with
    ## getting a table full of correlation relations within the users
        
    
    # Selecting users that have actually rated the target item
    potential_neighbors = list(df_pred[df_pred[target_item].notnull()].index) ## ignore users which are not 
    ## correlated to active user
    neighbors = user_similarity.loc[target_user].filter(potential_neighbors) ## potential_neighbors as 
    ## boolean mask

    
    # Saving weights and Names of Top K users closest to the Target one
    closest_users = neighbors.sort_values(ascending=False)[:top_k]
    
    sum_weight = closest_users.sum() # weight-normalization term
    target_rating_bias = df.loc[target_user].mean()

    predicted_score = target_rating_bias # start from user's own bias

    for user in closest_users.keys():
        user_rating_bias = df.loc[user].mean()

        user_weight = (closest_users[user]/sum_weight)
        user_rating_dev = df.loc[user, target_item] - user_rating_bias

        predicted_score += user_weight * user_rating_dev # take other ratings into account, apply weighting
        ## same as on slide pdf page 20
    
    
    return predicted_score

# quick check
user_based_CF(df, target_user = target_user, target_item = target_item)

4.658975830340907

## Item-Based CF
### What do we predict?

In [8]:
# Set User-Item to predict score
target_user = 'Active_User'
target_item = 'ACDC'

print("<"+target_user + "> <" + target_item + "> previous value: " + str(df.loc[target_user,target_item]))

# Create a copy of the data to mess with it
df_pred = df.copy(deep = True)

# Set the score to predict to NaN for convenience (It may have beem NaN already)
df_pred.loc[target_user,target_item] = np.nan
df_pred

<Active_User> <ACDC> previous value: nan


,RHCP,Radiohead,Clown_Core,Pink_Floyd,ACDC
user_1,3.0,NaN,2.0,3.0,3.0
user_2,4.0,3.0,4.0,3.0,NaN
user_3,3.0,2.0,1.0,4.0,4.0
user_4,NaN,5.0,4.0,3.0,1.0
Active_User,5.0,NaN,3.0,4.0,NaN


### Finding top K Similar Items

In [9]:
# Calculating Item-similarity

df_adjusted = df.apply(lambda x: x - x.mean(), axis=1) ## x.mean() refers to average rating of individual user
## axis=1 refers to the rows/users of the df
## x describes each entry value
## we now get a dataframe where each entry = user?_rating_item?? - user?_mean 
## --> needed for adjusted cosine similarity!!

item_mask = df_adjusted.loc[target_user].notnull() # items rated by target_user
item_mask[target_item] = True # explicitly add target item there for convenience ## Because we want to 
## make a prediction for the target item which was not rated by active user so far
## In contrast, other items which were not rated by the active shall be deleted

user_mask = df_adjusted[target_item].notnull() # users who rated target_item


# Interaction matrix prepared for adjusted cosine similarity calculation
df_adjusted = df_adjusted.loc[user_mask, item_mask]
## user_mask: delete users/rows who not rated target item
## item_mask: delete items/columns which were not rated by active user

res = pd.Series(dtype=float) # array to be filled with cosine similarities between the target and other users!!

for item in df_adjusted.columns: ## all column names/items (target item inclusive) which are relevant for
    ## determining the target location
    if not(item == target_item):
        res[item] = cosine_similarity(df_adjusted[[item, target_item]].dropna().T)[0,1]
        ## get cosine similarity between one relevant item and the target item
        
# taking two most similar items
top_k_items = 2
closest_items = res.sort_values(ascending=False)[:top_k_items]

closest_items


RHCP          0.770826
Pink_Floyd    0.644237
dtype: float64

## Predicting the rating

In [10]:
# Predicting the rating
# My solution:
sum_similars = closest_items.sum() ## sum up similarity over k-nearest items
enumerator = 0

for index, single_item_simi_value in enumerate(closest_items):
    title_cur_item = closest_items.keys()[index]
    rating_active_user_cur_item = df_pred[title_cur_item][target_user]
    enumerator += single_item_simi_value * rating_active_user_cur_item
    
final_itembased_prediction = enumerator/sum_similars

final_itembased_prediction
# ... something is missing here....

4.5447291009838064

## <font color='red'>TASK 2/4</font>: Item-Based CF
Complete the above code and compose it into a function. Consult lecture slides to make sure you don't miss a thing! Use the following template:

In [11]:
def item_based_CF(df: pd.DataFrame, target_user: str, target_item: str, top_k = 2) -> float:
    
    
    '''
    df - Data Frame - interation matrix
    target_user - String - user to predict rating for
    target_item - String - item to predict rating for
    top_k - Int - number of neighbors to predict ratings from (default 2)
    
    returns - float - predicted rating 
    '''
    # Calculating Item-similarity

    df_adjusted = df.apply(lambda x: x - x.mean(), axis=1) ## x.mean() refers to average rating of individual user
    ## axis=1 refers to the rows/users of the df
    ## x describes each entry value
    ## we now get a dataframe where each entry = user?_rating_item?? - user?_mean 
    ## --> needed for adjusted cosine similarity!!

    item_mask = df_adjusted.loc[target_user].notnull() # items rated by target_user
    item_mask[target_item] = True # explicitly add target item there for convenience ## Because we want to 
    ## make a prediction for the target item which was not rated by active user so far
    ## In contrast, other items which were not rated by the active shall be deleted

    user_mask = df_adjusted[target_item].notnull() # users who rated target_item


    # Interaction matrix prepared for adjusted cosine similarity calculation
    df_adjusted = df_adjusted.loc[user_mask, item_mask]
    ## user_mask: delete users/rows who not rated target item
    ## item_mask: delete items/columns which were not rated by active user

    res = pd.Series(dtype=float) # array to be filled with cosine similarities between the target and other users!!

    ## Compute the cosine_similarity:
    for item in df_adjusted.columns: ## all column names/items (target item inclusive) which are relevant for
        ## determining the target location
        if not(item == target_item):
            res[item] = cosine_similarity(df_adjusted[[item, target_item]].dropna().T)[0,1]
            ## get cosine similarity between one relevant item and the target item

    # taking two most similar items
    closest_items = res.sort_values(ascending=False)[:top_k]

    
    # My added part to compute the final prediction:
    
    sum_similars = closest_items.sum() ## sum up similarity over k-nearest items
    enumerator = 0

    for index, single_item_simi_value in enumerate(closest_items):
        title_cur_item = closest_items.keys()[index]
        rating_active_user_cur_item = df_pred[title_cur_item][target_user]
        enumerator += single_item_simi_value * rating_active_user_cur_item

    return float(enumerator/sum_similars) ## the final item based prediction
    # error was here!!! Very important to use float()because in first line by function initalization:
    # '-> float'


# quick check
print(f'Prediction for top_k = 15 as given by jupyter notebook:\n{item_based_CF(df, target_user = target_user, target_item = target_item, top_k = 15)}\n') 

print(f'Prediction for top_k = 2 as given by the function:\n{item_based_CF(df, target_user = target_user, target_item = target_item)}')

Prediction for top_k = 15 as given by jupyter notebook:
6.305436935363652

Prediction for top_k = 2 as given by the function:
4.5447291009838064


## <font color='red'>TASK 3/4</font>: Closer to the Real World
In real world situations recommender systems may face all kinds of extreme circumstances. For example there may be not enough data to make a prediction.
Revise and improve the functions you have written for Tasks 1 and 2.

In particular address the following issues:
* If there is no suitable neighbors - return user bias as a prediction;
* If it is impossible to calculate user bias - return average rating: **3**;
* The functions should not corrupt input data;
* The functions should act sensibly when given top_k argument from range [1, +inf];
* The functions should not take into account users / items with zero or negative similarity to the target;

In [12]:
# go back to slides and look what is missing

# calculate every value in the table and then calculate the rmse

In [13]:
## For user_based prediction:
## Something new is marked as '## NEW'
def user_based_CF(df: pd.DataFrame, target_user: str, target_item: str, top_k = 2) -> float:
    '''
    df - Data Frame - interation matrix
    target_user - String - user to predict rating for
    target_item - String - item to predict rating for
    top_k - Int - number of neighbors to predict ratings from (default 2)
    
    returns - float - predicted rating 
    '''
    
    ## NEW: 
    ## Target user hasn't done a rating so far --> return a 3 as prediction
    if df.loc[target_user].isnull().all():
        return 3
    

    # Create a copy of the data to mess with it
    df_pred = df.copy(deep = True) 

    # Set the score to predict to NaN for convenience (It may have beem NaN already)
    df_pred.loc[target_user,target_item] = np.nan
    
    
    # Calculating similarities between users
    user_similarity = df_pred.T.corr(method='pearson')
    ## NEW comment: there has to be at least one rating of a user when he/she shall be considered in the
    ## similarity. Therefore, the mean of user rating (user bias) also exists and can be computed.
    
    
    # Selecting users that have actually rated the target item
    potential_neighbors = list(df_pred[df_pred[target_item].notnull()].index)
    
    neighbors = user_similarity.loc[target_user].filter(potential_neighbors) 
    
    ## NEW: Let's say that top_k shall be maximal 49. That should be enough to make meaningful predictions.
    if top_k >= 50:
        top_k = 49 ## 49 is the closest valid choice to k >= 50; good k typically in praxis: 20 < k < 50
        
    elif top_k < 10 and top_k < len(neighbors):
        if len(neighbors) < 50:
            k = len(neighbors)
        else:
            k = 21 ## NEW: a good k is in practice bigger than 20. Furthermore, 21 is closest valid k choice 
            ## to the inputted k
    

    closest_users = neighbors.sort_values(ascending=False)[:top_k]
    
    
    ## NEW:
    if len(closest_items) == 0:
        return df.mean(axis=1)[target_user]
    
    closest_users_reduced = closest_users[closest_users > 0] ## NEW: we only consider users which have a
    ## higher similarity value than 0
    
    target_rating_bias = df.loc[target_user].mean()
    
    ## NEW: When there exists no similar user to target user we simply return the target user's mean/bias
    if len(closest_users_reduced) == 0:
        return target_rating_bias 
    

    predicted_score = target_rating_bias # start from user's own bias
    sum_weight = closest_users_reduced.sum() # weight-normalization term

    for user in closest_users_reduced.keys(): ## NEW: We make same procedure as before but this time with
        ## closest users which have a similarity > 0
        
        
        user_rating_bias = df.loc[user].mean()               
        
        user_weight = (closest_users_reduced[user]/sum_weight)
        user_rating_dev = df.loc[user, target_item] - user_rating_bias

        predicted_score += user_weight * user_rating_dev
    
    
    return predicted_score

# quick check
user_based_CF(df, target_user = target_user, target_item = target_item)

4.658975830340907

In [14]:
## For item_based prediction:

def item_based_CF(df: pd.DataFrame, target_user: str, target_item: str, top_k = 2) -> float:
    '''
    df - Data Frame - interation matrix
    target_user - String - user to predict rating for
    target_item - String - item to predict rating for
    top_k - Int - number of neighbors to predict ratings from (default 2)
    
    returns - float - predicted rating 
    ''' 
    
    ## NEW: Target user hasn't done a rating so far --> return a 3 as prediction
    if df.loc[target_user].isnull().all():
        return 3
    
    # Calculating Item-similarity
    df_adjusted = df.apply(lambda x: x - x.mean(), axis=1) 

    item_mask = df_adjusted.loc[target_user].notnull() # items rated by target_user
    item_mask[target_item] = True # explicitly add target item there for convenience 

    user_mask = df_adjusted[target_item].notnull() # users who rated target_item


    # Interaction matrix prepared for adjusted cosine similarity calculation
    df_adjusted = df_adjusted.loc[user_mask, item_mask]

    res = pd.Series(dtype=float) 

    ## Compute the cosine_similarity:
    for item in df_adjusted.columns: 
        ## determining the target location
        if not(item == target_item):
            try: ## NEW: cosine_smilarity only works when there is a item to compare with.
                res[item] = cosine_similarity(df_adjusted[[item, target_item]].dropna().T)[0,1]
                ## NEW comment: we only calculate similarities from df_adjusted, i. e. the user biases are
                ## computeable here; furthermore, we don't need elsewhere the user bias
            except: 
                return df.mean(axis=1)[target_user]

            
    ## NEW: Let's say that top_k shall be maximal 49. That should be enough to make meaningful predictions.
    if top_k >= 50:
        top_k = 49 ## 49 is the closest valid choice to k >= 50; good k typically in praxis: 20 < k < 50
        
    elif top_k < 10 and top_k < len(res):
        if len(res) < 50:
            k = len(res)
        else:
            k = 21 ## NEW: a good k is in practice bigger than 20. Furthermore, 21 is closest valid k choice 
            ## to the inputted k
    ## Remark: The choice of k above is valid for user_based CF, I claim that we can also use these values
    ## for item_based CF.
              
    # taking two most similar items
    closest_items = res.sort_values(ascending=False)[:top_k] ## NEW comment: when top_k is too large, we
    ## simply include all res-values here and will select the fitting ones in the next step

    
    ## NEW:
    if len(closest_items) == 0:
        return df.mean(axis=1)[target_user]
        
    sum_similars = 0 ## NEW
    enumerator = 0
    counter = 0 ## NEW

    for index, single_item_simi_value in enumerate(closest_items):
        
        ## NEW:
        if single_item_simi_value <= 0: ## the item doesn't fit for our prediction
            counter += 1
        else: # Only items which fit to the searched one shall be considered
            sum_similars += single_item_simi_value 
            
            title_cur_item = closest_items.keys()[index]
            rating_active_user_cur_item = df_pred[title_cur_item][target_user]
            enumerator += single_item_simi_value * rating_active_user_cur_item

    ## New:    
    if counter == len(closest_items): ## no neighbour is fitting --> return user bias as prediction
        return df.mean(axis=1)[target_user] ## the user bias is the mean value of the given ratings of a user

        
    return enumerator/sum_similars ## the final item based prediction


# quick check
print(f'Prediction for top_k = 15 as given by jupyter notebook:\n{item_based_CF(df, target_user = target_user, target_item = target_item, top_k = 15)}\n') 

print(f'Prediction for top_k = 2 as given by the function:\n{item_based_CF(df, target_user = target_user, target_item = target_item)}')

Prediction for top_k = 15 as given by jupyter notebook:
4.5447291009838064

Prediction for top_k = 2 as given by the function:
4.5447291009838064


---

## Recap: Evaluation I
|        | item_1 | item_2 | item_3 | item_4 | item_5 |
| ---    |   ---  |   ---  |   ---  |   ---  |   ---  |
| user_1 | 3 |   | 2 | 3 | 3 |
| user_2 | 4 | 3 | 4 | 3 |   |
| user_3 | 3 | 2 | 1 | 4 | 4 |
| user_4 |   | 5 | 4 | 3 | 1 |
| user_5 | 5 |   | 3 | 4 | **(?)** |

* How do we know if predictions given by our systems are any good? How do we compare systems to each other?
* We have been predicting ratings for unseen items so far. What if we predict known ratings and compare the result to the actual values?
* Let us take all known ratings as test set;

$T$ - Test set\
$u$ - User\
$i$ - Item\
$r_{u,i}$ - Ground truth rating for the user $u$ and the item $i$\
$r'_{u,i}$ - Predicted rating for the user $u$ and the item $i$

### Mean Absolute Error (MAE)

* literal approach: aggregate differences between predictions and actual values.

$MAE = \frac{1}{|T|}\sum \limits _{(u,i)\in T}{|r'_{u,i} - r_{u,i}|}$

**What is worse: if we make two mistakes by 1 star, or one mistake by 2 stars**?\
**MAE doesn't care, but you should!**

| Item_id | Actual Values | System 1 | System 2 |
| - |      :---      |    ---   |    ---   |
|1|⭐️⭐️⭐️⭐️| 4 | 4 |
|2|⭐️⭐️⭐️⭐️⭐️ | <font color='red'>**4**</font> | 5 |
|3|⭐️⭐️⭐️⭐️ | <font color='red'>**5**</font> | <font color='red'>**2**</font> |

### Root Mean Squared Error (RMSE)

* RMSE pays more attention to more prominent deviations;

$RMSE = \sqrt{\frac{1}{|T|}\sum \limits _{(u,i)\in T}{(r'_{u,i} - r_{u,i})^2}}$

| Metric | System 1 | System 2 |
| :---   |    ---   |    ---   |
| MAE | 0.67 | 0.67 |
| RMSE |  0.82 | 1.15 |

* Can we normalize RMSE (to [0, 1])? How do we calculate max RMSE? What normalized RMSE can be good for?

## <font color='red'>TASK 4/4</font>: RMSE
Write a function that can be used to evaluate CF techniques from the first part of the exercise. The function needs to take a DataFrame and a callable name and return a single float value. **No normalization required!** See template below.

Normalization is good to compare over different types of ratings the RMSE values. To the internet normalization is possible for RMSE.

In [15]:
def rmse(TS: pd.DataFrame, rec_function, top_k=2)->float:
    '''
    TS - Data Frame - interation matrix with NaNs for unknown values
    rec_function - collaborative filtering function to evaluate (see example calls at the very bottom of the cell)
    top_k - Int - top k users or items to infer from
    
    returns - float - RMSE value 
    '''    
    # Now, I am totally seeing what you meant by 'at this place df is wrong'. 
    # It is rather obvious. I only was too stressed to get it. :D
    # So, thank you for the second chance.
    
    series_target_users = TS.T.keys()
    series_target_items = TS.keys()
    counter_not_nan = 0 ## is |T|
    rmse_sum = 0
    
    for single_target_user in series_target_users:
        for single_target_item in series_target_items:
            
            true_rating = TS.loc[single_target_user,single_target_item]
            
            if np.isnan(true_rating) == False: ## Ignore all slots which are NaN
                counter_not_nan += 1
                predicted_rating = rec_function(TS, single_target_user, single_target_item, top_k)
                rmse_sum += (predicted_rating - true_rating)**2

    return (rmse_sum/counter_not_nan)**(0.5) ## = RMSE

    
# quick check
print(rmse(df, item_based_CF), rmse(df, user_based_CF))

1.0213167223785722 1.2199081371708123


In [16]:
# Check-list:
# Task 1
print("UB-CF returned "+str(user_based_CF(df, target_user = target_user, target_item = target_item)))
# Task 2
print("IB-CF returned "+str(item_based_CF(df, target_user = target_user, target_item = target_item)))
# Task 3 - no separate function to write
# Task 4
print("RMSE4UB-CF returned "+ str(rmse(df, user_based_CF, 3)))
print("RMSE4IB-CF returned "+ str(rmse(df, item_based_CF, 3)))

UB-CF returned 4.658975830340907
IB-CF returned 4.5447291009838064
RMSE4UB-CF returned 1.2199081371708123
RMSE4IB-CF returned 1.0213167223785722


In [17]:
# Leave this cell the way it is, please.